<a href="https://colab.research.google.com/github/praveentecsl/High-Engagement-Content-Classification-model/blob/main/SocialMedia_engagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**mount to drive**



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**MODEL**

=====================================================================================================

###Data Engineering & Preprocessing

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('ad_campaign_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'ad_campaign_data.csv'

In [ ]:
print(df.head()) #commit for testing

  user_id  age  gender  location  device_type   ad_id  ad_category  \
0  U25795   45       0         2            2  A77161            2   
1  U10860   29       0         0            2  A41167            1   
2  U86820   59       0         1            2  A55186            1   
3  U64886   47       0         1            2  A59722            4   
4  U16265   36       1         0            0  A38955            2   

   impressions  clicks  conversions  ...  tfidf_11 tfidf_12  tfidf_13  \
0         2460    1223            1  ...       0.0      0.0       0.0   
1          162     117            1  ...       0.0      0.0       0.0   
2         3648     554            1  ...       0.0      0.5       0.5   
3          648     259            1  ...       0.0      0.5       0.5   
4         3598    1860            1  ...       0.0      0.5       0.5   

   tfidf_14  tfidf_15  tfidf_16  tfidf_17  tfidf_18  tfidf_19  tfidf_20  
0       0.0       0.0  0.000000       0.0   0.57735       0.0  0.0

create a new block testing

1.  praveen chandeepa weerasinghe
2.   List item testing



clustering for reaching

==========================================================================================================

#**********git configuration**********

##setup and pull

In [ ]:
# --- SECTION 1: SETUP & SYNC ---
import os
from google.colab import drive, userdata

# 1. Mount Google Drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("drive already mounted")

# 2. Configuration (Ensure 'GITHUB_TOKEN' is in your Colab Secrets 🔑)
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_USER = "praveentecsl"
REPO_NAME = "High-Engagement-Content-Classification-model"
# Navigate to where you want to store the project in Drive
PROJECT_PATH = f"/content/drive/MyDrive/{REPO_NAME}"

# 3. Clone or Pull Logic
if not os.path.exists(PROJECT_PATH):
    # If the folder doesn't exist, go to Drive root and clone
    %cd /content/drive/MyDrive/
    !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git
    %cd {REPO_NAME}
    print("✅ Repository Cloned for the first time.")
else:
    # If folder exists, go inside and pull latest changes from collaborator
    %cd {PROJECT_PATH}
    !git pull origin main
    print("✅ Repository updated with latest changes (Git Pull).")

# 4. Identity Configuration for Commits
!git config --global user.email "your-email@example.com"
!git config --global user.name "Praveen"

drive already mounted
/content/drive/MyDrive/High-Engagement-Content-Classification-model


##push code

In [ ]:
!pwd

/content/drive/MyDrive/High-Engagement-Content-Classification-model


In [ ]:
!git status


Refresh index: 100% (4/4), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   SocialMedia_engagement.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
#@title 🚀 Git Push Interface
#@markdown Enter your message in the box on the right, then run this cell.

# 1. Capture the input from the form into a variable
Commit_Message = "initiallize preprocessing part" #@param {type:"string"}

import os

# 2. Ensure we are in the Repo directory
%cd {PROJECT_PATH}

# 3. Git Workflow
print(f"--- Processing: {Commit_Message} ---")

# Add all changes
!git add .

# Commit using the variable from the form
# We use the '$' prefix to pass the Python variable into the shell command
!git commit -m "{Commit_Message}"

# Push to GitHub
!git push https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git main

print(f"\n✅ Successfully pushed to GitHub with message: {Commit_Message}")

/content/drive/MyDrive/High-Engagement-Content-Classification-model
--- Processing: initiallize preprocessing part ---
[main 173a145] initiallize preprocessing part
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite SocialMedia_engagement.ipynb (92%)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 819 bytes | 117.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/praveentecsl/High-Engagement-Content-Classification-model.git
   e95c7f7..173a145  main -> main

✅ Successfully pushed to GitHub with message: initiallize preprocessing part
